# IVDA Group 10 - Ikealytics 
## Group Assignment 2

- Michael Sigg 18-618-843
- Noah Croes 24-738-759
- Leon Keutzer 24-736-001
- Jasur Fayzullaev 23-748-973

In [2]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# import data